In [3]:
import pandas as pd
import numpy as np
import pickle
import haversine as hs
import requests
import zipfile
from bs4 import BeautifulSoup
from io import BytesIO, StringIO
from urllib.request import urlopen

# <center>dataset creation

## read crop data from offical data source

In [4]:
# read crop yield data
ert_raw = pd.read_csv('secondary_data/41241-01-03-4_flat.csv', sep=';', encoding='iso8859_15')

# rename labels
ert = ert_raw[['Zeit', '1_Auspraegung_Code', '2_Auspraegung_Label', 'ERT001__Hektarertraege__dt/ha']].copy()
ert.columns = ['year', 'district', 'crop', 'yield']

In [5]:
# drop all district codes with less than 5 digits
# all other represent higher administrative regions

drop = np.unique([x for x in ert['district'] if (len(x)<5)])
drop_index = []
for d in drop:
    drop_index.append(ert[ert['district'] == d].index)
drop_index = np.array(drop_index)
drop_index = drop_index.flatten()

ert.drop(index=drop_index, inplace=True)

# replace missing values with nan
# replace comma with dots
ert['yield'].replace({'.': np.nan, '-': np.nan, 'x': np.nan, '/': np.nan, '...': np.nan}, inplace=True)
ert['yield'] = ert['yield'].str.replace(',', '.')
ert['yield'] = pd.to_numeric(ert['yield'])

# cropping Region_Code to 5 digits
ert['district'] = [x[:5] for x in ert['district']]

# creating some lists for later use
crops = ert['crop'].unique().tolist()
gmky5 = ert['district'].unique().tolist()


In [6]:
# writing in dictionary with structure data['district code]['crops']
data = {}
for district in gmky5:
    x = ert[ert['district']==district]
    if len(x)==220:    
        x = x.pivot(index='year', columns='crop', values='yield')
    else:
        x = x.groupby(['crop', 'year']).agg('mean').unstack('crop')['yield']
    crop = {}
    crop['crops'] = x
    data[district] = crop

## get geographial middle point of districts

In [7]:
path = 'secondary_data/'
files = [
    str(path+'31122007_Auszug_GV.xlsx'), str(path+'31122008_Auszug_GV.xlsx'), str(path+'31122009_Auszug_GV.xlsx'),
    str(path+'31122010_Auszug_GV.xlsx'), str(path+'31122011_Auszug_GV.xlsx'), str(path+'31122012_Auszug_GV.xlsx'),
    str(path+'31122013_Auszug_GV.xlsx'), str(path+'31122014_Auszug_GV.xlsx'), str(path+'31122015_Auszug_GV.xlsx'),
    str(path+'31122016_Auszug_GV.xlsx'), str(path+'31122017_Auszug_GV.xlsx'), str(path+'31122018_Auszug_GV.xlsx'),
    str(path+'31122019_Auszug_GV.xlsx'), str(path+'31122020_Auszug_GV.xlsx')
    ]

In [8]:
# reading excel files with coordinates of all municipalties,
# aggregating on district level by computing means of lat lon positions
dstrcts_ll = {}

for file, year in zip(files, np.arange(2006, 2021)):
        
    # load data
    df = pd.read_excel(file, sheet_name=1)

    df = df.iloc[:, [0, 2, 3, 4, 14, 15]]
    df.columns = ['Satz', 'k1', 'k2', 'k3', 'lon', 'lat']

    # cut upper and lower end
    cut_to = df.index[np.where(df['Satz']=='10')[0][0]]
    df.drop(np.arange(0, (cut_to+1)), inplace=True)

    try:
        cut_from = df.index[df['Satz'].isnull()][0]
        df.drop(np.arange(cut_from, (df.index[-1]+1)), inplace=True)
    except:
        pass

    # build GemKey5 through concat strings
    GemKey5 = [str(df.iloc[i, 1]) + str(df.iloc[i, 2]) + str(df.iloc[i, 3]) for i in np.arange(0, len(df))]
    df['GemKey5'] = GemKey5

    # mean values of lat and lon through districts
    try: 
        df['lat'] = pd.to_numeric(df['lat'].str.replace(',', '.'))
        df['lon'] = pd.to_numeric(df['lon'].str.replace(',', '.'))
    except:
        pass

    dx = df[['lat', 'lon']].groupby(df['GemKey5']).agg('mean').dropna()

    dstrcts_ll[year] = dx

In [9]:
# create dictionary with districts (keys) and lat lon
position = pd.DataFrame(columns=['lat', 'lon'])
for year in dstrcts_ll.keys():
    position = position.append(dstrcts_ll[year])
position = position.groupby(position.index).agg('mean')
dstrcts_ll = {district:[position.loc[district, 'lat'], position.loc[district, 'lon']] for district in position.index}

# add position to dataset
for gmky in data.keys():
    try:
        data[gmky]['position'] = dstrcts_ll[gmky]
    except:
        pass

## find near weather stations of districts

In [10]:
# read coordinates of weather stations from file
# source: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/annual/kl/historical/KL_Jahreswerte_Beschreibung_Stationen.txt

w_stations = pd.read_fwf('secondary_data/KL_Monatswerte_Beschreibung_Stationen.txt', 
                         colspecs=[(0, 5), (6, 14), (15, 23), (24, 38), (43, 49), (53, 60), (61, 100), (102, 123)], 
                         encoding='iso8859_15')

w_stations.columns=['Stations_id', 'von_datum', 'bis_datum', 'Stationshoehe', 
                    'geoBreite', 'geoLaenge', 'Stationsname', 'Bundesland']
w_stations.drop(0, axis=0, inplace=True)

w_stations['von_datum'] = pd.to_numeric(w_stations['von_datum'])
w_stations['bis_datum'] = pd.to_numeric(w_stations['bis_datum'])
w_stations['Stationshoehe'] = pd.to_numeric(w_stations['Stationshoehe'])
w_stations['geoBreite'] = pd.to_numeric(w_stations['geoBreite'])
w_stations['geoLaenge'] = pd.to_numeric(w_stations['geoLaenge'])

# keep only those weather stations providing data from 1999 onwards
w_stations = w_stations[w_stations['bis_datum']>19990000]

In [11]:
def find_close_weather_stations(latlon, radius):
    # center of district
    center_district = np.array(latlon)
    
    # looping through list of weather stations 
    distance = np.array([hs.haversine(center_district, np.array(w_stations.iloc[i, 4:6])) for i in range(len(w_stations))])
    distance = pd.Series(distance, index=w_stations['Stations_id'], name='distance')
    keep = distance<=radius
    distance = distance[keep]
        
    return(distance)

In [12]:
for district in data:
    print(f'district: {district}', end='\r')
    try:
        close_stations = find_close_weather_stations(data[str(district)]['position'], 30)
    except:
        close_stations = []
    data[str(district)]['close_stations'] = close_stations
print('done')

donerict: 16077


## retrieve data from weather stations for each district

In [13]:
# list of all stations needed
relevant_stations = [data[x]['close_stations'].index for x in data.keys()]

def list_index(x):
    try:
        y = x.values.tolist()
    except:
        y = []
    return y

relevant_stations = [list_index(x) for x in relevant_stations]
relevant_stations = [item for sublist in relevant_stations for item in sublist]
relevant_stations = np.unique(relevant_stations)

In [14]:
# path to DWD open data portal
path = 'https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/monthly/kl/historical/'

# read all files in directory and put them in a list
# weather data is provided with zip files
site_content = requests.get(path).text
soup = BeautifulSoup(site_content, 'html.parser').find_all('a')
files = [x.attrs['href'] for x in soup]

# Liste bereinigen: nur zip-Dateien
zipfiles = []
for i in files:
    if i[-3:] == 'zip':
        zipfiles.append(i)
    else:
        pass

# empty data frame for merging masked data
empty_df = pd.DataFrame(index=(np.arange(1999, 2021)), columns=(np.arange(1, 13)))

In [15]:
# this function opens the zip file of a given weather station (Station ID),
# extracts all relevant weather indicators, and returns them as dictionary

def get_indicators(statID):
    
    statID = '_' + statID + '_'        
    empty_df = pd.DataFrame(index=(np.arange(1999, 2021)), columns=(np.arange(1, 13)))

    for file in zipfiles:
        if (statID in file) == True:
            filepath = path+file
        else:
            pass

    try:
        filepath

        goal = urlopen(filepath)
        zippedfiles = zipfile.ZipFile(BytesIO(goal.read()))

        for i in zippedfiles.namelist():
            if ('produkt_klima_monat' in i) == True:
                datafile = i
            else:
                pass

        # unpack files and write in data frame
        data = zippedfiles.read(datafile).decode()
        df = pd.read_csv(StringIO(data), sep=';')
        df = df[df['MESS_DATUM_BEGINN']>19990000][['MESS_DATUM_BEGINN', 'MO_TT', 'MO_SD_S', 'MO_FK', 'MO_RR']]
        df = df[df['MESS_DATUM_BEGINN']<20210000]

        # get year and month
        df['year'] = [str(x)[0:4] for x in df['MESS_DATUM_BEGINN']]
        df['month'] = [str(x)[4:6] for x in df['MESS_DATUM_BEGINN']]
        df['year'] = pd.to_numeric(df['year'])
        df['month'] = pd.to_numeric(df['month'])

        df.drop(['MESS_DATUM_BEGINN'], axis=1, inplace=True)

        # missing values
        df.replace({-999: np.nan}, inplace=True)

        # bringing stuff together
        df_TT = df.pivot(index='year', columns='month', values='MO_TT')
        df_SD = df.pivot(index='year', columns='month', values='MO_SD_S')
        df_FK = df.pivot(index='year', columns='month', values='MO_FK')
        df_RR = df.pivot(index='year', columns='month', values='MO_RR')
        df_TT = empty_df.fillna(df_TT)
        df_SD = empty_df.fillna(df_SD)
        df_FK = empty_df.fillna(df_FK)
        df_RR = empty_df.fillna(df_RR)

        # as Dictionary
        station_indicators = {'TT': df_TT, 'SD': df_SD, 'FK': df_FK, 'RR': df_RR}
    
    except:
        station_indicators = {'TT': empty_df, 'SD': empty_df, 'FK': empty_df, 'RR': empty_df}

    
    return station_indicators

In [27]:
# if there is more than one weather station in a particular district,
# this functions comines the values

def combine_stations(stat_ids):
    
    station_ids = stat_ids
    station_values = {}

    #if more than one weather station in district
    if len(station_ids) > 1:

        for i in station_ids:
            station_values[i] = get_indicators(i)

        TT_list = []
        SD_list = []
        FK_list = []
        RR_list = []

        for i in station_values.keys():
            TT_list = TT_list + [station_values[i]['TT']]
            SD_list = SD_list + [station_values[i]['SD']]
            FK_list = FK_list + [station_values[i]['FK']]
            RR_list = RR_list + [station_values[i]['RR']]

        raw_df_TT = pd.concat(TT_list)
        raw_df_SD = pd.concat(SD_list)
        raw_df_FK = pd.concat(FK_list)
        raw_df_RR = pd.concat(RR_list)

        empty_df = pd.DataFrame(index=(np.arange(1999, 2021)), columns=(np.arange(1, 13)))

        df_TT_full = empty_df.copy()
        df_SD_full = empty_df.copy()
        df_FK_full = empty_df.copy()
        df_RR_full = empty_df.copy()

        for i in empty_df.index:
            df_TT_full.loc[i] = raw_df_TT.loc[i].mean(axis=0)
            df_SD_full.loc[i] = raw_df_SD.loc[i].mean(axis=0)
            df_FK_full.loc[i] = raw_df_FK.loc[i].mean(axis=0)
            df_RR_full.loc[i] = raw_df_RR.loc[i].mean(axis=0)
    
    #if just one weather station in district
    elif len(stat_ids)==0:
        empty_df = pd.DataFrame(index=(np.arange(1999, 2021)), columns=(np.arange(1, 13)))

        df_TT_full = empty_df
        df_SD_full = empty_df
        df_FK_full = empty_df
        df_RR_full = empty_df
    
    else:
        _ = get_indicators(station_ids[0])
        df_TT_full = _['TT']
        df_SD_full = _['SD']
        df_FK_full = _['FK']
        df_RR_full = _['RR']

    return {'TT':df_TT_full, 'SD':df_SD_full, 'FK':df_FK_full, 'RR':df_RR_full}  

<i>caution: the execution of the following cell will take some time!</i>

In [28]:
# retreive weather data from stations near district center

errors = []

for i, district in enumerate(list(data.keys())):
    print(f'district: {district} ({i+1}/{len(data)})', end='\r')
    try:
        stations = data[district]['close_stations'].index.tolist()
        weather_data = combine_stations(stations)
    except:
        weather_data = []
        errors.append(district)
    
    data[district]['weather_data'] = weather_data

print(f'Errors on districts {errors}')

Errors on districts ['11001', '11002', '11003', '11004', '11005', '11006', '11007', '11008', '11009', '11010', '11011', '11012', '15101', '15151', '15153', '15154', '15159', '15171', '15202', '15256', '15260', '15261', '15265', '15266', '15268', '15303', '15352', '15355', '15357', '15358', '15362', '15363', '15364', '15367', '15369', '15370']


## get Soil Moisture Index (SMI)

In [29]:
import netCDF4 as nc
from pyproj import Transformer
from geopy.geocoders import Nominatim

# empty data frame to fill later
empty_df = pd.DataFrame(index=(np.arange(1999, 2021)), columns=(np.arange(1, 13)))

# reading UFZ data set
ds_ob = nc.Dataset('secondary_data/248980_SMI_SM_L02_Oberboden_monatlich_1951-2020_inv.nc')
ds_gb = nc.Dataset('secondary_data/248981_SMI_SM_Lall_Gesamtboden_monatlich_1951-2020_inv.nc')

# UFZ data is grid data
# transform from lat/lon (EPSG:4326) to Gauß-Krüger Zone 4 (EPSG: 31468)
transformer = Transformer.from_crs(4326, 31468)

In [30]:
# this function get the smi value for a given latitude and longitude
# ds = UFZ dataset

def get_smi (ds, lat, lon):
    # transform
    nor, eas = transformer.transform(lat, lon)
    
    # find nearest point
    near_north = min(ds['northing'][:], key=lambda x:abs(x-nor))
    near_east = min(ds['easting'][:], key=lambda x:abs(x-eas))

    # get index
    ix_northing = list(ds['northing']).index(near_north)
    ix_easting = list(ds['easting']).index(near_east)

    #read SMIi for region
    smi_raw = ds['SMI'][:, ix_northing, ix_easting].data

    # sort list by years
    smi_years = np.reshape(smi_raw, [70, 12])

    # set year as index for data frame
    smi = pd.DataFrame(smi_years, columns=[np.arange(1, 13)], index=np.arange(1951, 2021))

    return smi

In [31]:
# add smi data frame to district dictionary

errors = []
for district in data.keys():
    print(district, end='\r')
    try:
        lat, lon = data[district]['position']
        smi_ob = get_smi(ds_ob, lat, lon).loc[1999:,:]
        smi_gb = get_smi(ds_gb, lat, lon).loc[1999:,:]
    except:
        smi_ob, smi_gb = [], []
        errors.append(district)
    
    data[district]['SMI_OB'] = smi_ob
    data[district]['SMI_GB'] = smi_gb

print(f'\nErrors on districts: {errors}')

16077
Errors on districts: ['11001', '11002', '11003', '11004', '11005', '11006', '11007', '11008', '11009', '11010', '11011', '11012', '15101', '15151', '15153', '15154', '15159', '15171', '15202', '15256', '15260', '15261', '15265', '15266', '15268', '15303', '15352', '15355', '15357', '15358', '15362', '15363', '15364', '15367', '15369', '15370']


create objects for better handling

In [32]:
from harvestingtime import district

In [33]:
dataset = {}

for distr in list(data.keys()):
    try:
        print(f'district: {distr}', end='\r')
        dataset[distr] = district(data[distr])
    except:
        pass
print('\ndone')

district: 16077
done


In [34]:
with open('data/dataset.pkl', 'wb') as file:
    pickle.dump(dataset, file)